In [2]:
import sys
sys.path.append("../")

In [3]:
# statement_name = "../data/Справка_о_движении_денежных_средств_0819.pdf"
# incomes = 9678606.76
# expences = -9128682.41
# bank_name = "tbank"

statement_name = "../data/ozonbank_document_6732671.pdf"
incomes = 94948.0
expences = -100459
bank_name = "ozon"



# 0. Настройка

In [4]:
from src.finman.utils.pdf import extract_text_from_pdf
from src.finman.expenses import tbank, ozon
from src.finman.utils.gsheet import GSheetWorker

In [5]:
sheet_id = "1GqwNPuOtQUXfltK6F4oHpx-U1BqBgiDZfC5YfDcJyto"
gcreds_file = "../secrets/finman-433017-da308d823497.json"

gsw = GSheetWorker(gcreds_file)

In [6]:
if bank_name == "tbank":
    trans_col = "Сумма операции в валюте карты"
    date_col = "Дата операции"
    time_col = "Время операции"
    spreadsheet_name = "t-bank (main)"
    parse_operations = tbank.parse_operations

elif bank_name == "ozon":
    trans_col = "Сумма операции"
    date_col = "Дата операции"
    time_col = "Время операции"
    spreadsheet_name = "ozon-bank (main)"
    parse_operations = ozon.parse_operations

# 1. Прочитать выписку

In [7]:
text = extract_text_from_pdf(statement_name)
transactions_df = parse_operations(text)

In [8]:
transactions_df.head()

,id,Дата операции,Время операции,Сумма операции,Документ,Назначение платежа
0,4281248653,2024-10-30,2024-10-30 20:28:29,-3704.0,2650036929,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
1,390c1475e2,2024-10-30,2024-10-30 20:27:23,4000.0,2650029559,Зачисление по переводу денежных средств по кар...
2,6d65919f51,2024-10-28,2024-10-28 21:07:20,-1568.0,2636552813,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
3,36e94095e2,2024-10-28,2024-10-28 21:07:04,1000.0,2636551588,Зачисление по переводу денежных средств по кар...
4,c0ee681d97,2024-10-17,2024-10-17 13:11:54,-563.0,2542381115,"Оплата товаров/услуг на Платформе Ozon, заказ ..."


In [9]:
assert transactions_df.loc[transactions_df[trans_col] > 0][trans_col].sum() == incomes
assert transactions_df.loc[transactions_df[trans_col] < 0][trans_col].sum() == expences

# 2. Запроцессить гугол

In [10]:
gs_df = gsw.get_df(sheet_id, spreadsheet_name)

In [11]:
gs_df.head()

,id,Дата операции,Время операции,Сумма операции,Документ,Назначение платежа,Категория,Подкатегория,Хэштэг,Долг,Комментарий
0,4348aedc63,"16.08, Fri",2024-08-16 18:21,-482,2125895187,"Оплата товаров/услуг на Платформе Ozon, заказ ...",Household,Accessories,None,None,None
1,7725e5161f,"13.08, Tue",2024-08-13 19:02,938,2107392458,"Возврат оплаты за товары/услуги, купленные на ...",Bank,Cashback,,,Возврат покупки
2,8e154900e2,"13.08, Tue",2024-08-13 9:24,-2682,2103792680,"Оплата товаров/услуг на Платформе Ozon, заказ ...",Household,Accessories,None,None,None
3,71275ad89c,"13.08, Tue",2024-08-13 9:24,2690,1280184534,Перевод A4226062454289120000000011310501 через...,Transfer,None,None,None,None
4,ead5097e9a,"12.08, Mon",2024-08-12 10:29,-2871,2097210103,"Оплата товаров/услуг на Платформе Ozon, заказ ...",Household,Accessories,None,None,None


In [12]:
missing_recrods = transactions_df.loc[~transactions_df['id'].isin(gs_df['id'])]

In [13]:
missing_recrods.head()

,id,Дата операции,Время операции,Сумма операции,Документ,Назначение платежа
0,4281248653,2024-10-30,2024-10-30 20:28:29,-3704.0,2650036929,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
1,390c1475e2,2024-10-30,2024-10-30 20:27:23,4000.0,2650029559,Зачисление по переводу денежных средств по кар...
2,6d65919f51,2024-10-28,2024-10-28 21:07:20,-1568.0,2636552813,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
3,36e94095e2,2024-10-28,2024-10-28 21:07:04,1000.0,2636551588,Зачисление по переводу денежных средств по кар...
4,c0ee681d97,2024-10-17,2024-10-17 13:11:54,-563.0,2542381115,"Оплата товаров/услуг на Платформе Ozon, заказ ..."


In [14]:
missing_recrods[date_col] = missing_recrods[date_col].astype(str)
missing_recrods[time_col] = missing_recrods[time_col].dt.strftime('%Y-%m-%d %H:%M')

gsw.insert_df(missing_recrods, sheet_id, spreadsheet_name)

/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_65378/3544399752.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_recrods[date_col] = missing_recrods[date_col].astype(str)
/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_65378/3544399752.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_recrods[time_col] = missing_recrods[time_col].dt.strftime('%Y-%m-%d %H:%M')
